In [47]:
from IPython.display import HTML
HTML('''
    <style> body {font-family: "Roboto Condensed Light", "Roboto Condensed";} h2 {padding: 10px 12px; background-color: #E64626; position: static; color: #ffffff; font-size: 40px;} .text_cell_render p { font-size: 15px; } .text_cell_render h1 { font-size: 30px; } h1 {padding: 10px 12px; background-color: #E64626; color: #ffffff; font-size: 40px;} .text_cell_render h3 { padding: 10px 12px; background-color: #0148A4; position: static; color: #ffffff; font-size: 20px;} h4:before{ 
    content: "@"; font-family:"Wingdings"; font-style:regular; margin-right: 4px;} .text_cell_render h4 {padding: 8px; font-family: "Roboto Condensed Light"; position: static; font-style: italic; background-color: #FFB800; color: #ffffff; font-size: 18px; text-align: center; border-radius: 5px;}input[type=submit] {background-color: #E64626; border: solid; border-color: #734036; color: white; padding: 8px 16px; text-decoration: none; margin: 4px 2px; cursor: pointer; border-radius: 20px;}</style>
    <script> code_show=true; function code_toggle() {if (code_show){$('div.input').hide();} else {$('div.input').show();} code_show = !code_show} $( document ).ready(code_toggle);</script>
    <form action="javascript:code_toggle()"><input type="submit" value="Hide/show all code."></form>
''')

# Sydney Liveability Analysis

## Group - 5  F10Adv-01

In [48]:
import pandas as pd
import geopandas as gpd
import json
from shapely.geometry import Point, Polygon, MultiPolygon
from geoalchemy2 import Geometry, WKTElement
import matplotlib.pyplot as plt

In [49]:
sa2_data = gpd.read_file("SA2_2016_AUST/SA2_2016_AUST.shp")

In [50]:
break_and_enter = gpd.read_file("break_and_enter/BreakEnterDwelling_JanToDec2021.shp")

In [51]:
school_catchments_future = gpd.read_file("school_catchments/catchments_future.shp")

In [52]:
school_catchments_primary = gpd.read_file("school_catchments/catchments_primary.shp")

In [53]:
school_catchments_secondary = gpd.read_file("school_catchments/catchments_secondary.shp")

In [54]:
neighbourhoods = pd.read_csv('Neighbourhoods.csv')

In [55]:
BusinessStats = pd.read_csv('BusinessStats.csv')

In [56]:
Playgrounds_json = json.load(open('Playgrounds.geojson'))

Playgrounds_DataFrame = pd.DataFrame(Playgrounds_json["features"])
Playgrounds_pandas_data=[]
for index, row in Playgrounds_DataFrame.iterrows():
    append_row = [row["geometry"]["coordinates"][0],row["geometry"]["coordinates"][1],row["properties"]["Name"]]
    Playgrounds_pandas_data.append(append_row)
    
Playgrounds = pd.DataFrame(Playgrounds_pandas_data, columns = ['X', 'Y','Name'])
Playgrounds['geom'] = gpd.points_from_xy(Playgrounds.X, Playgrounds.Y)  # creating the geometry column
Playgrounds = Playgrounds.drop(columns=['X', 'Y'])  # removing the old latitude/longitude fields

In [57]:
Lights_json = json.load(open('Lights.geojson'))
Lights_DataFrame = pd.DataFrame(Lights_json["features"])
Lights_pandas_data=[]
for index, row in Lights_DataFrame.iterrows():
    append_row = [row["geometry"]["coordinates"][0],row["geometry"]["coordinates"][1],row["properties"]["Location"]]
    Lights_pandas_data.append(append_row)
    
Lights = pd.DataFrame(Lights_pandas_data, columns = ['X', 'Y','Location'])
Lights['geom'] = gpd.points_from_xy(Lights.X, Lights.Y)  # creating the geometry column
Lights = Lights.drop(columns=['X', 'Y'])  # removing the old latitude/longitude fields

In [58]:
sa2_data = sa2_data.drop(sa2_data[(sa2_data['SA4_NAME16'].str.contains("No usual address", case=False)) |
                                  (sa2_data['SA4_NAME16'].str.contains("Migratory - Offshore - Shipping", case=False)) |
                         sa2_data['SA3_NAME16'].str.contains("No usual address", case=False) |
                         sa2_data['SA3_NAME16'].str.contains("Migratory - Offshore - Shipping", case=False) |
                         sa2_data['SA2_NAME16'].str.contains("No usual address", case=False) | 
                         sa2_data['SA2_NAME16'].str.contains("Migratory - Offshore - Shipping", case=False) | 
                         sa2_data['GCC_NAME16'].str.contains("No usual address", case=False) | 
                         sa2_data['GCC_NAME16'].str.contains("Migratory - Offshore - Shipping", case=False) |
                         sa2_data['GCC_NAME16'].str.contains("Rest of", case=False)].index
                        )

sa2_data = sa2_data[sa2_data['GCC_NAME16']=='Greater Sydney'] 

In [59]:
# neighbourhoods.isnull().values.any()
# nan_rows = neighbourhoods[neighbourhoods.isnull().T.any()]
# # print(nan_rows)
neighbourhoods = neighbourhoods.drop(neighbourhoods[neighbourhoods.isnull().T.any()].index)
# neighbourhoods.get_value(319, 'population', takeable=False)
# for index, row in nan_rows.iterrows():
#     print(row)
# print(type(nan_rows))


In [60]:
for index, row in neighbourhoods.iterrows():
    old_value = row['population']
    old_value_split = str(row['population']).split(",")
    new_value = ""
    for i in old_value_split :
        new_value = new_value+i
    neighbourhoods['population'] = neighbourhoods['population'].replace([old_value],int(new_value))

#     print(row['c1'], row['c2'])

# neighbourhoods['population'] = neighbourhoods['population'].replace(['old value'],'new value')

In [61]:
from sqlalchemy import create_engine
import psycopg2
import psycopg2.extras
import json

credentials = "Credentials.json"

def pgconnect(credential_filepath, db_schema="public"):
    with open(credential_filepath) as f:
        db_conn_dict = json.load(f)
        host       = db_conn_dict['host']
        db_user    = db_conn_dict['user']
        db_pw      = db_conn_dict['password']
        default_db = db_conn_dict['user']
        try:
            db = create_engine('postgresql+psycopg2://'+db_user+':'+db_pw+'@'+host+'/'+default_db, echo=False)
            conn = db.connect()
            print('Connected successfully.')
        except Exception as e:
            print("Unable to connect to the database.")
            print(e)
            db, conn = None, None
        return db,conn

def query(conn, sqlcmd, args=None, df=True):
    result = pd.DataFrame() if df else None
    try:
        if df:
            result = pd.read_sql_query(sqlcmd, conn, params=args)
        else:
            result = conn.execute(sqlcmd, args).fetchall()
            result = result[0] if len(result) == 1 else result
    except Exception as e:
        print("Error encountered: ", e, sep='\n')
    return result

In [62]:
db, conn = pgconnect(credentials)

Connected successfully.


In [63]:
query(conn, "select PostGIS_Version()")

,postgis_version
0,3.0 USE_GEOS=1 USE_PROJ=1 USE_STATS=1


In [64]:
srid = 4326
Playgrounds['geom'] = Playgrounds['geom'].apply(lambda x: WKTElement(x.wkt, srid=srid))
Lights['geom'] = Lights['geom'].apply(lambda x: WKTElement(x.wkt, srid=srid))

In [65]:
sa2_data = sa2_data[['SA2_MAIN16', 'SA2_NAME16','SA3_NAME16','geometry']].copy()
break_and_enter = break_and_enter[['OBJECTID', 'Density','geometry']].copy()
school_catchments_future = school_catchments_future[['USE_ID','CATCH_TYPE', 'USE_DESC','geometry']].copy()
school_catchments_primary = school_catchments_primary[['USE_ID', 'USE_DESC','geometry']].copy()
school_catchments_secondary = school_catchments_secondary[['USE_ID', 'USE_DESC','geometry']].copy()
neighbourhoods = neighbourhoods[['area_id','land_area','population', 'number_of_dwellings','number_of_businesses','median_annual_household_income','avg_monthly_rent','0-4','5-9','10-14','15-19']].copy()
BusinessStats = BusinessStats[['area_id','number_of_businesses','accommodation_and_food_services','retail_trade','agriculture_forestry_and_fishing','health_care_and_social_assistance','public_administration_and_safety','transport_postal_and_warehousing']].copy()
Lights = Lights[['Location', 'geom']].copy()
Playgrounds = Playgrounds[['Name', 'geom']].copy()

In [66]:
def create_wkt_element(geom, srid):
    if geom!=None :
        if geom.geom_type == 'Polygon':
            geom = MultiPolygon([geom])
        return WKTElement(geom.wkt, srid)

# sa2_data_og = sa2_data.copy()  # creating a copy of the original for later
sa2_data['geom'] = sa2_data['geometry'].apply(lambda x: create_wkt_element(geom=x,srid=srid))  # applying the function
sa2_data = sa2_data.drop(columns="geometry")  # deleting the old copy

# break_and_enter_og = break_and_enter.copy()  # creating a copy of the original for later
break_and_enter['geom'] = break_and_enter['geometry'].apply(lambda x: create_wkt_element(geom=x,srid=srid))  # applying the function
break_and_enter = break_and_enter.drop(columns="geometry")  # deleting the old copy

# school_catchments_future_og = school_catchments_future.copy()  # creating a copy of the original for later
school_catchments_future['geom'] = school_catchments_future['geometry'].apply(lambda x: create_wkt_element(geom=x,srid=srid))  # applying the function
school_catchments_future = school_catchments_future.drop(columns="geometry")  # deleting the old copy

# school_catchments_primary_og = school_catchments_primary.copy()  # creating a copy of the original for later
school_catchments_primary['geom'] = school_catchments_primary['geometry'].apply(lambda x: create_wkt_element(geom=x,srid=srid))  # applying the function
school_catchments_primary = school_catchments_primary.drop(columns="geometry")  # deleting the old copy

# school_catchments_secondary_og = school_catchments_secondary.copy()  # creating a copy of the original for later
school_catchments_secondary['geom'] = school_catchments_secondary['geometry'].apply(lambda x: create_wkt_element(geom=x,srid=srid))  # applying the function
school_catchments_secondary = school_catchments_secondary.drop(columns="geometry")  # deleting the old copy

In [128]:
school_catchments_primary

,USE_ID,USE_DESC,geom
0,2838,Parklea PS,MULTIPOLYGON (((150.93563850416 -33.7161211047...
1,3943,Gordon WPS,MULTIPOLYGON (((151.140850430121 -33.744626792...
2,2404,Lindfield EPS,MULTIPOLYGON (((151.1833640465581 -33.74748398...
3,4393,Carlingford WPS,MULTIPOLYGON (((151.0451821055135 -33.77303212...
4,4615,Caddies Ck PS,MULTIPOLYGON (((150.9256732797658 -33.72960300...
...,...,...,...
1661,4383,E A Southee PS,MULTIPOLYGON (((147.9462089946497 -34.55863148...
1662,3275,Tumbarumba PS,MULTIPOLYGON (((148.1288534897749 -35.60081845...
1663,2239,Jindera PS,MULTIPOLYGON (((146.8614794320412 -35.87511069...
1664,3594,Louth PS,MULTIPOLYGON (((145.1840275468519 -29.65804989...


In [68]:
# conn.execute("""
# CREATE SCHEMA IF NOT EXISTS Sydney_Liveability_Analysis;
# SET search_path TO Sydney_Liveability_Analysis;
# """)

# sa2_data
conn.execute("""
DROP TABLE IF EXISTS sa2_data;
CREATE TABLE sa2_data (
    "SA2_MAIN16" INTEGER,
    "SA2_NAME16" VARCHAR(80),
    "SA3_NAME16" VARCHAR(80),
    "geom" GEOMETRY(MULTIPOLYGON,4326)
);"""
)

# neighbourhood
conn.execute("""
DROP TABLE IF EXISTS neighbourhood;
CREATE TABLE neighbourhood (
    "area_id" INTEGER,
    "land_area" NUMERIC,
    "population" INTEGER,
    "number_of_dwellings" VARCHAR(80),
    "number_of_businesses" INTEGER,
    "median_annual_household_income" NUMERIC,
    "avg_monthly_rent" INTEGER,
    "0-4" INTEGER,
    "5-9" INTEGER,
    "10-14" INTEGER,
    "15-19" INTEGER
);"""
)

# break and enter
conn.execute("""
DROP TABLE IF EXISTS break_and_enter;
CREATE TABLE break_and_enter (
    "OBJECTID" INTEGER,
    "Density" VARCHAR(80),
    "geom" GEOMETRY(MULTIPOLYGON,4326)
);"""
)

# school_catchments_future
conn.execute("""
DROP TABLE IF EXISTS school_catchments_future;
CREATE TABLE school_catchments_future (
    "USE_ID" INTEGER, 
    "CATCH_TYPE" VARCHAR(80), 
    "USE_DESC" VARCHAR(80),
    "geom" GEOMETRY(MULTIPOLYGON,4326)
    
);"""
)

# school_catchments_primary
conn.execute("""
DROP TABLE IF EXISTS school_catchments_primary;
CREATE TABLE school_catchments_primary (
    "USE_ID" INTEGER,
    "USE_DESC" VARCHAR(80),
    "geom" GEOMETRY(MULTIPOLYGON,4326)
    
);"""
)

# school_catchments_secondary
conn.execute("""
DROP TABLE IF EXISTS school_catchments_secondary;
CREATE TABLE school_catchments_secondary (
    "USE_ID" INTEGER,
    "USE_DESC" VARCHAR(80),
    "geom" GEOMETRY(MULTIPOLYGON,4326)
    
);"""
)


# BusinessStats
conn.execute("""
DROP TABLE IF EXISTS businessstats;
CREATE TABLE businessstats (
    "area_id" INTEGER,
    "number_of_businesses" INTEGER,
    "accommodation_and_food_services" INTEGER,
    "retail_trade" INTEGER,
    "agriculture_forestry_and_fishing" INTEGER,
    "health_care_and_social_assistance" INTEGER,
    "public_administration_and_safety" INTEGER,
    "transport_postal_and_warehousing" INTEGER
);"""
)

# Playgrounds
conn.execute("""
DROP TABLE IF EXISTS playgrounds;
CREATE TABLE playgrounds (
   "Name" VARCHAR(200),
   geom GEOMETRY(POINT, 4326)
);"""
)

# Lights
conn.execute("""
DROP TABLE IF EXISTS lights;
CREATE TABLE lights (
   "Location" VARCHAR(200),
   geom GEOMETRY(POINT, 4326)
);"""
)


In [69]:
sa2_data.to_sql('sa2_data', conn, if_exists='append', index=False, dtype={'geom': Geometry('MULTIPOLYGON', srid)})

In [70]:
neighbourhoods.to_sql('neighbourhood', conn, if_exists='append', index=False)

In [71]:
break_and_enter.to_sql("break_and_enter", conn, if_exists='append', index=False, dtype={'geom': Geometry('MULTIPOLYGON', srid)})

In [72]:
school_catchments_future.to_sql("school_catchments_future", conn, if_exists='append', index=False, dtype={'geom': Geometry('MULTIPOLYGON', srid)})

In [73]:
school_catchments_primary.to_sql("school_catchments_primary", conn, if_exists='append', index=False, dtype={'geom': Geometry('MULTIPOLYGON', srid)})

In [74]:
school_catchments_secondary.to_sql("school_catchments_secondary", conn, if_exists='append', index=False, dtype={'geom': Geometry('MULTIPOLYGON', srid)})

In [75]:
BusinessStats.to_sql('businessstats', conn, if_exists='append', index=False)

In [76]:
Lights.to_sql('lights', conn, if_exists='append', index=False, dtype={'geom': Geometry('POINT', srid)})

In [77]:
Playgrounds.to_sql('playgrounds', conn, if_exists='append', index=False, dtype={'geom': Geometry('POINT', srid)})

In [ ]:
# query(conn, "select * from sa2_data") # 312 Rows
# query(conn, "select * from neighbourhood") # 322 Rows
# query(conn, "select * from break_and_enter") # 2594 Rows
# query(conn, "select * from school_catchments_future") # 43 Rows
# query(conn, "select * from school_catchments_primary") # 1666 Rows
# query(conn, "select * from school_catchments_secondary") # 435 Rows
# query(conn, "select * from businessstats") # 2301 Rows
# query(conn, "select * from lights") # 8433 Rows
# query(conn, "select * from playgrounds") # 151 Rows

In [135]:
sql="""

with Health AS 
(SELECT avg(1000.0*b."health_care_and_social_assistance"/n."population") AS mean,
stddev(1000.0*b."health_care_and_social_assistance"/n."population") AS sd
FROM businessstats b, neighbourhood n
WHERE b."area_id" = n."area_id"),

Retail AS
(SELECT avg(1000.0*b."retail_trade"/n."population") AS mean,
stddev(1000.0*b."retail_trade"/n."population") AS sd
FROM businessstats b, neighbourhood n
WHERE b."area_id" = n."area_id"),

Accom AS
(SELECT avg(1000.0*b."accommodation_and_food_services"/n."population") AS mean,
stddev(1000.0*b."accommodation_and_food_services"/n."population") AS sd
FROM businessstats b, neighbourhood n
WHERE b."area_id" = n."area_id")

SELECT s."SA2_NAME16", b."health_care_and_social_assistance", n."population", 
abs((1000.0*b."health_care_and_social_assistance"/n."population")-Health.mean)/Health.sd AS HZ,
abs((1000.0*b."retail_trade"/n."population")-Retail.mean)/Retail.sd AS RZ,
abs((1000.0*b."accommodation_and_food_services"/n."population")-Accom.mean)/Accom.sd AS AZ

FROM sa2_data s, businessstats b, neighbourhood n, Health, Retail, Accom
WHERE s."SA2_MAIN16" = b."area_id"
AND b."area_id" = n."area_id"
ORDER BY rz desc
;
"""

query(conn,sql)

,SA2_NAME16,health_care_and_social_assistance,population,hz,rz,az
0,Sydney - Haymarket - The Rocks,1217,31010,6.874785,9.310323,13.737354
1,Chullora,11,1225,0.584661,8.657748,2.066460
2,Badgerys Creek,0,62,1.281562,8.078197,1.042036
3,Castle Hill - Central,90,7992,1.058858,2.005274,0.557525
4,North Sydney - Lavender Bay,261,11819,3.307951,1.905175,4.066902
...,...,...,...,...,...,...
294,Guildford - South Granville,108,22367,0.278049,0.016675,0.251605
295,Kingsgrove (North) - Earlwood,119,25211,0.300574,0.015770,0.069198
296,Woolaware - Burraneer,53,8260,0.051967,0.011508,0.120024
297,Newtown - Camperdown - Darlington,360,27450,1.444069,0.003502,0.949610


In [58]:
conn.close()
db.dispose()